## Basical settings

Import packages

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.modeling import models
from astropy.modeling import functional_models
from astropy import units as u
import os, json, random, cv2
from PIL import Image
from numpyencoder import NumpyEncoder
import multiprocessing
import time
import torch

import utils

# for testing purpose
import importlib

plt.rcParams["figure.figsize"] = (15,10)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [5]:
# !nvidia-smi
print(torch.__version__)
print(torch.cuda.is_available())

1.13.1
True


Global variables

In [6]:
wavelengths = np.arange(400, 800)*u.nm
FWHM        = 0.5
lambda_Na   = 589.0*u.nm
lambda_Mg   = 518.3*u.nm
width       = 1000
height      = 1500

## Generate dataset using multiprocessing: 

In [8]:
def generate_singledata(image_id): 
    '''
    '''
    filename = str(image_id) + "_" + time.strftime("%Y%m%d-%H%M%S") + ".jpg"
    print('generating image: %s...'%filename)
    # parameters
    num_stars       = 200
    x_stars         = (np.random.rand(num_stars)*(width-10*FWHM)+5*FWHM).astype(np.int16)
    y_stars         = (np.random.rand(num_stars)*(height-10*FWHM)+5*FWHM).astype(np.int16)
    # TODO: change the border where the meteor can show
    num_meteors     = 20
    length_meteors  = (np.random.rand(num_meteors)*500).astype(np.int16)                      # pixel
    x_meteors       = (np.random.rand(num_meteors)*(width-10*FWHM)+5*FWHM).astype(np.int16)
    y_meteors       = (np.random.rand(num_meteors)*(height-10*FWHM)+5*FWHM).astype(np.int16)
    # image - parameters needed for `generate_image` function
    coord_stars     = np.array([x_stars, y_stars]).T            # [N, 2]pixel
    amp_stars       = np.random.rand(num_stars)*1e8             # [N]erg -> pixel value
    T_stars         = (np.random.rand(num_stars)*15000+5000)*u.K# [N]5000K-20000K
    coord_meteors   = np.array([x_meteors, y_meteors]).T        # [n, 2]pixel
    amp_meteors     = np.random.rand(num_meteors)*1e8           # [n]erg -> pixel value
    T_meteors       = [2000*u.K]*num_meteors                    # [n]5000K-20000K
    angle_meteors   = np.random.rand(num_meteors)*2*np.pi       # [n]radian
    angle_slit      = np.random.rand()*2*np.pi                  # radian
    length_slit     = 100
    # generate image
    image = utils.generate_image(coord_stars, amp_stars, T_stars, coord_meteors, amp_meteors, T_meteors, 
                            angle_meteors, length_meteors, angle_slit, length_slit)
    image8 = image.astype(np.uint8)-1
    im = Image.fromarray(image8)
    im.save(os.path.join(dir, filename), quality=100)
    # generate label
    lock.acquire()
    labels = json.load(open(os.path.join(dir, 'labels.json'), 'r'))
    # label - image
    label_image = utils.annotate_image(filename, image_id)
    labels['images'].append(label_image)
    # label - annotation - meteor
    for [x_meteor, y_meteor], angle_meteor, length_meteor in zip(coord_meteors, angle_meteors, length_meteors):
        label_meteor = utils.annotate_meteor(x_meteor, y_meteor, angle_slit, length_slit, angle_meteor, 
                                                length_meteor, anno_id_shared.value, image_id)
        labels['annotations'].append(label_meteor)
        anno_id_shared.value += 1
    # label - annotation - stars
    for x, y in coord_stars: 
        label_star = utils.annotate_star(x, y, angle_slit, length_slit, anno_id_shared.value, image_id)
        if label_star != None: 
            labels['annotations'].append(label_star)
            anno_id_shared.value += 1
    # rewrite json file everytime finishing generating an image
    with open(os.path.join(dir, 'labels.json'), 'w') as f:
        json.dump(labels, f, sort_keys=False, indent=2, separators=(',', ': '), cls=NumpyEncoder)
    lock.release()

In [12]:
dir='./dataset/'
os.makedirs(dir, exist_ok=True)

labels = utils.blank_label()
with open(os.path.join(dir, 'labels.json'), 'w') as f:
    json.dump(labels, f, sort_keys=False, indent=2, separators=(',', ': '), cls=NumpyEncoder)

num_images = 1000


anno_id_start = 1# max([i['id'] for i in labels['annotations']]) + 1
anno_id = anno_id_start
image_id_start = 1# max([i['id'] for i in labels['images']]) + 1

manager = multiprocessing.Manager()
anno_id_shared = manager.Value('i', anno_id)
lock = multiprocessing.RLock()
args = zip(range(image_id_start, image_id_start + num_images))
with multiprocessing.Pool(processes=4) as pool:
    results = pool.starmap(generate_singledata, args)

generating image: 64_20230306-083858.jpg...generating image: 1_20230306-083858.jpg...generating image: 127_20230306-083858.jpg...


generating image: 190_20230306-083858.jpg...
generating image: 2_20230306-083928.jpg...
generating image: 191_20230306-083928.jpg...
generating image: 65_20230306-083929.jpg...
generating image: 128_20230306-083929.jpg...
generating image: 3_20230306-083950.jpg...
generating image: 66_20230306-083952.jpg...
generating image: 192_20230306-083954.jpg...
generating image: 129_20230306-083955.jpg...
generating image: 67_20230306-084017.jpg...
generating image: 4_20230306-084018.jpg...
generating image: 193_20230306-084019.jpg...
generating image: 130_20230306-084020.jpg...
generating image: 68_20230306-084036.jpg...
generating image: 5_20230306-084036.jpg...
generating image: 194_20230306-084038.jpg...
generating image: 131_20230306-084038.jpg...
generating image: 69_20230306-084100.jpg...
generating image: 6_20230306-084102.jpg...
generating image: 195_202303

In [14]:
importlib.reload(utils)

<module 'utils' from '/mnt/c/Users/Chisen/OneDrive - University of Illinois - Urbana/Research/Meteor spectrum/Metspec/utils.py'>